In [1]:
from pytorch_unet import *
from torch import nn
import pandas as pd
from timerit import Timerit
from IPython.display import display, HTML

In [2]:
print(torch.__version__)

1.9.0+cu102


In [3]:
model = UNet(128,2).eval()

x = torch.randn((2,1,128,128))
pred = model(x)

In [5]:
device_name = " "
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(torch.cuda.current_device())
else:
    device_name = 'cpu'
    
print('Using device:', device_name)

device = " "
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

Using device: GeForce RTX 2060 SUPER


In [6]:
metric_labels = [
    "Frames",
    "# Samples",
    "First PT (s)",
    "Avg PT (s)",
    "Min PT (s)",
    "Max PT (s)",
    "Std. Dev. (s)",
]
    
print('Using device:', device_name)

results_df = pd.DataFrame(columns = metric_labels)
frames = [2, 10, 20, 30, 40, 50]
for l in range(len(frames)):

    model = UNet(128,2)
    if device == 'cuda':
        model = model.to(torch.device('cuda'))
    t = Timerit(num = 100, verbose=0)
    
    model = model.eval()
    
    for timer in t:
        x = torch.randn((frames[l],1,128,128))
        with timer:
            x = x.to(device)
            pred = model(x)
    
    mf_times = t.times
    
    results_df.loc[l] = [
        frames[l],
        100,
        mf_times[0],
        int((np.mean(mf_times)*100000)) / 100000,
        np.min(mf_times),
        np.max(mf_times),
        int((np.std(mf_times)*100000)) / 100000
    ]

print('Pytorch Multi-Frame Batch Prediction Timings')
print('"PT" = Prediction Time')
display(results_df)

Using device: GeForce RTX 2060 SUPER
Pytorch Multi-Frame Batch Prediction Timings
"PT" = Prediction Time


,Frames,# Samples,First PT (s),Avg PT (s),Min PT (s),Max PT (s),Std. Dev. (s)
0,2.0,100.0,0.008630,0.00268,0.002566,0.008630,0.00060
1,10.0,100.0,0.004234,0.00272,0.002657,0.004234,0.00016
2,20.0,100.0,0.003520,0.00283,0.002741,0.003520,0.00014
3,30.0,100.0,0.004117,0.00302,0.002933,0.004151,0.00019
4,40.0,100.0,0.004141,0.00308,0.003016,0.004141,0.00013
5,50.0,100.0,0.004253,0.00319,0.003122,0.004253,0.00013
